In [13]:
from keras import layers
from keras import models
from keras.preprocessing.image import ImageDataGenerator

In [14]:
def preprocess_input(x):
    x /= 255.0
    x -= 0.5
    x *= 2.0
    return x

In [15]:
train_dir = 'E:/Jupyter NB/10 class Classification/Train_val_test/train'
val_dir = 'E:/Jupyter NB/10 class Classification/Train_val_test/val'
#test_dir = 'E:/Jupyter NB/10 class Classification/Train_val_test/test'

In [16]:
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(299, 299, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(20, activation='softmax'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 297, 297, 32)      896       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 148, 148, 32)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 146, 146, 64)      18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 73, 73, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 71, 71, 128)       73856     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 35, 35, 128)       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 33, 33, 128)      

In [17]:

data_generator = ImageDataGenerator(
    data_format='channels_last',
    preprocessing_function=preprocess_input
)
'''
data_generator = ImageDataGenerator(
    rotation_range=30, 
    zoom_range=0.3,
    horizontal_flip=True, 
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.001,
    channel_shift_range=0.1,
    fill_mode='reflect',
    data_format='channels_last',
    preprocessing_function=preprocess_input
)
#val_datagen = ImageDataGenerator(rescale=1./255)
#test_datagen = ImageDataGenerator(rescale=1./255)
'''
train_generator = data_generator.flow_from_directory(
    train_dir,
    target_size=(299, 299),
    batch_size=20,
    shuffle=False)

validation_generator = data_generator.flow_from_directory(
    val_dir,
    target_size=(299, 299),
    batch_size=20,
    shuffle=False)

'''
test_generator = data_generator.flow_from_directory(
    test_dir,
    target_size=(299, 299),
    batch_size=20,
    shuffle=False)
'''

Found 2000 images belonging to 20 classes.
Found 400 images belonging to 20 classes.


'\ntest_generator = data_generator.flow_from_directory(\n    test_dir,\n    target_size=(299, 299),\n    batch_size=20,\n    shuffle=False)\n'

# Model Compilation

In [18]:
from keras.losses import categorical_crossentropy as logloss
from keras import backend as K

def loss(y_true, y_pred):
    entropy = -K.mean(K.sum(y_pred*K.log(y_pred), 1))
    beta = 0.1
    return logloss(y_true, y_pred) - beta*entropy

In [19]:
from keras import optimizers

model.compile(loss=loss,
    optimizer=optimizers.SGD(lr=1e-2),
    metrics=['categorical_crossentropy', 'accuracy'])

# Training

In [20]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=118,
    epochs=40,
    validation_data=validation_generator,
    validation_steps=30)

Epoch 1/40
118/118 [==============================] - 41s 350ms/step - loss: 2.7227 - categorical_crossentropy: 3.0216 - accuracy: 0.0284 - val_loss: 2.6981 - val_categorical_crossentropy: 2.9848 - val_accuracy: 0.0867
Epoch 2/40
118/118 [==============================] - 67s 566ms/step - loss: 2.6796 - categorical_crossentropy: 2.9783 - accuracy: 0.0746 - val_loss: 2.1257 - val_categorical_crossentropy: 2.9115 - val_accuracy: 0.1383
Epoch 3/40
118/118 [==============================] - 67s 570ms/step - loss: 2.6679 - categorical_crossentropy: 2.9646 - accuracy: 0.0822 - val_loss: 2.3772 - val_categorical_crossentropy: 2.8481 - val_accuracy: 0.1083
Epoch 4/40
118/118 [==============================] - 77s 653ms/step - loss: 2.5928 - categorical_crossentropy: 2.8826 - accuracy: 0.1254 - val_loss: 2.6003 - val_categorical_crossentropy: 2.7553 - val_accuracy: 0.1517
Epoch 5/40
118/118 [==============================] - 67s 567ms/step - loss: 2.5449 - categorical_crossentropy: 2.8303 - acc

118/118 [==============================] - 67s 567ms/step - loss: 0.0027 - categorical_crossentropy: 0.0044 - accuracy: 0.9987 - val_loss: 5.3607 - val_categorical_crossentropy: 4.6208 - val_accuracy: 0.4050
Epoch 39/40
118/118 [==============================] - 67s 567ms/step - loss: 0.0023 - categorical_crossentropy: 0.0037 - accuracy: 0.9987 - val_loss: 5.4232 - val_categorical_crossentropy: 5.1535 - val_accuracy: 0.3617
Epoch 40/40
118/118 [==============================] - 67s 566ms/step - loss: 0.0023 - categorical_crossentropy: 0.0039 - accuracy: 0.9987 - val_loss: 5.3458 - val_categorical_crossentropy: 4.6893 - val_accuracy: 0.4100


In [21]:
model.save_weights('teacher_weights.hdf5')

In [22]:
'''
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
test_generator.reset()
pred=model.predict_generator(test_generator,
steps=STEP_SIZE_TEST,
verbose=1)
'''

'\nSTEP_SIZE_TEST=test_generator.n//test_generator.batch_size\ntest_generator.reset()\npred=model.predict_generator(test_generator,\nsteps=STEP_SIZE_TEST,\nverbose=1)\n'

In [23]:
'''
STEP_SIZE_TEST=train_generator.n//train_generator.batch_size
train_generator.reset()
pred=model.predict_generator(train_generator,
    steps=STEP_SIZE_TEST,
    verbose=1)
'''

'\nSTEP_SIZE_TEST=train_generator.n//train_generator.batch_size\ntrain_generator.reset()\npred=model.predict_generator(train_generator,\n    steps=STEP_SIZE_TEST,\n    verbose=1)\n'